In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.tools

In [22]:
df.describe()

,Year,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,Polio,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing,Life_expectancy
count,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000,2864.000000
mean,2007.500000,30.363792,42.938268,192.251775,4.820882,84.292598,77.344972,25.032926,86.499651,86.271648,0.894288,11540.924930,36.675915,4.865852,4.899825,7.632123,0.206704,0.793296,68.856075
std,4.610577,27.538117,44.569974,114.910281,3.981949,15.995511,18.659693,2.193905,15.080365,15.534225,2.381389,16934.788931,136.485867,4.438234,4.525217,3.171556,0.405012,0.405012,9.405608
min,2000.000000,1.800000,2.300000,49.384000,0.000000,12.000000,10.000000,19.800000,8.000000,16.000000,0.010000,148.000000,0.080000,0.100000,0.100000,1.100000,0.000000,0.000000,39.400000
25%,2003.750000,8.100000,9.675000,106.910250,1.200000,78.000000,64.000000,23.200000,81.000000,81.000000,0.080000,1415.750000,2.097500,1.600000,1.600000,5.100000,0.000000,1.000000,62.700000
50%,2007.500000,19.600000,23.100000,163.841500,4.020000,89.000000,83.000000,25.500000,93.000000,93.000000,0.150000,4217.000000,7.850000,3.300000,3.400000,7.800000,0.000000,1.000000,71.400000
75%,2011.250000,47.350000,66.000000,246.791375,7.777500,96.000000,93.000000,26.400000,97.000000,97.000000,0.460000,12557.000000,23.687500,7.200000,7.300000,10.300000,0.000000,1.000000,75.400000
max,2015.000000,138.100000,224.900000,719.360500,17.870000,99.000000,99.000000,32.100000,99.000000,99.000000,21.680000,112418.000000,1379.860000,27.700000,28.600000,14.100000,1.000000,1.000000,83.800000


In [11]:
def linear_regression(data, minimal=False, custom=None):

    #Feature Engineering Function
    def feature_engineering(df):
        df = df.copy()
        df = pd.get_dummies(df, columns=["Region"], drop_first=True, prefix="region", dtype=int)        #One Hot Encoding the region column
        df["GDP_per_capita"] = df["GDP_per_capita"] / 1000                                              #Convert GDP into easier to use units
        df = sm.add_constant(df)
        return df
    
    #Reading the data
    df = pd.read_csv(data)

    #Setting up variables and target
    fcols = list(df.columns)
    fcols.remove("Life_expectancy")
    fcols.remove("Country")
    X = df[fcols]
    y = df.Life_expectancy

    #Train-test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

    #Applying Feature Engineering Function
    X_train_fe = feature_engineering(X_train)

    #Specifying columns to use in case minimal version is used
    if custom:
        cols = custom
    elif minimal:
        cols = ['GDP_per_capita', 'region_Central America and Caribbean',
       'region_Middle East', 'region_South America', 'region_Asia', 'region_Rest of Europe', 'region_Oceania',
       'region_European Union', 'region_North America', 'Schooling', 'const']
    else:
        cols = ['region_Central America and Caribbean', 'region_South America',
       'Under_five_deaths', 'GDP_per_capita', 'region_Oceania',
       'region_European Union', 'Schooling', 'Incidents_HIV', 'const']

    #Creating and fitting the model
    lin_reg = sm.OLS(y_train, X_train_fe[cols])
    results = lin_reg.fit()

    #Using model to make predictions on training data and report Root Mean Squared Error
    y_pred = results.predict(X_train_fe[cols])
    train_rmse = statsmodels.tools.eval_measures.rmse(y_train, y_pred)
    print(f"Training data RMSE:\t{train_rmse}")

    #Feature Engineering on test data to match training
    X_test_fe = feature_engineering(X_test)

    #Predicting on testing data and reporting RMSE
    y_test_pred = results.predict(X_test_fe[cols])
    test_rmse = statsmodels.tools.eval_measures.rmse(y_test, y_test_pred)
    print(f"Testing data RMSE:\t{test_rmse}")

    return results


In [23]:
results = linear_regression("Life Expectancy Data.csv", custom=cols_hanaa) 
results.summary()

Training data RMSE:	1.4253607687371903
Testing data RMSE:	1.402238000966035


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 1.952e+04
Date:                Wed, 29 May 2024   Prob (F-statistic):               0.00
Time:                        11:28:03   Log-Likelihood:                -4062.8
No. Observations:                2291   AIC:                             8138.
Df Residuals:                    2285   BIC:                             8172.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   80.3713      0.084    961.959      0.000      80.207      80.535
Adult_mortality                         -0.0447      0.000    -99.559      0.000      -0.046      -0.044
Under_five_deaths                       -0.0884      0.001    -77.075      0.000      -0.091      -0.086
GDP_per_capita                           0.0506      0.002     23.415      0.000       0.046       0.055
region_Central America and Caribbean     1.2566      0.101     12.480      0.000       1.059       1.454
region_European Union                    1.0767      0.095     11.309      0.000       0.890       1.263
==============================================================================
Omnibus:                       38.047   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.918
Skew:                          -0.041   Prob(JB):                     2.42e-16
Kurtosis:                       3.864   Cond. No.                         871.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
list(results.params)

[53.29586723292625,
 0.12243668415386055,
 0.928052406587341,
 8.456219692528556,
 10.938777802504932,
 10.662916648072365,
 11.117855051281126,
 10.434409816790037,
 7.733166882339386,
 8.775122941806782,
 11.260807581924542]

In [20]:
vif_cols = ['Under_five_deaths', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America', 'alcohol_exp', 'const']

step_cols = ['Adult_mortality', 'Economy_status_Developed', 'region_Central America and Caribbean', 'region_South America', 'Under_five_deaths', 'GDP_per_capita', 'region_Oceania','region_European Union', 'Schooling', 'BMI', 'Year', 'Incidents_HIV', 'Hepatitis_B', 'const']

step_cols_vif = ['region_Central America and Caribbean', 'region_South America',
       'Under_five_deaths', 'GDP_per_capita', 'region_Oceania',
       'region_European Union', 'Schooling', 'Incidents_HIV', 'const']

cols = ['const', 'Year', 'Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America', 'gdp_per_capita_exp',
       'alcohol_exp']

cols_ethical = ['const', 'Year', 'Alcohol_consumption', 'GDP_per_capita',
       'Population_mln', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America']

cols_ethical_stepwise = ['Schooling', 'const', 'GDP_per_capita', 'region_Central America and Caribbean', 'region_Middle East', 'region_South America', 'region_Asia', 'Economy_status_Developed', 'Economy_status_Developing', 'region_Rest of Europe', 'region_Oceania', 'Year', 'region_European Union', 'region_North America', 'Alcohol_consumption', 'Population_mln']

cols_ethical_vif = ['GDP_per_capita', 'Population_mln', 'Economy_status_Developing',
       'region_Asia', 'region_Central America and Caribbean',
       'region_European Union', 'region_Middle East', 'region_North America',
       'region_Oceania', 'region_Rest of Europe', 'region_South America', 'const']

cols_ethical_stepwise_vif = ['GDP_per_capita', 'region_Central America and Caribbean',
       'region_Middle East', 'region_South America', 'region_Asia',
       'Economy_status_Developing', 'region_Rest of Europe', 'region_Oceania',
       'region_European Union', 'region_North America', 'Population_mln']

cols_sumye = ['const',
       'Under_five_deaths', 
       'Adult_mortality',
       'GDP_per_capita',
        'Population_mln'
               ]

cols_hanaa = ['const','Adult_mortality','Under_five_deaths',
       'GDP_per_capita','region_Central America and Caribbean', 'region_European Union']

cols_ethical_sumye = ['const',
       'GDP_per_capita',
       'Schooling',
       'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America'
               ]